## Ollama

拟借助 Ollama 搭建本地大模型，使用 open-webui 进行 RAG 操作。

阅读 ollama 文档，预计可使用以下功能：
- Generate a completion
- Generate Embeddings

关于 Ollama 的使用，这里纠正一个误区。`ollama run model` 命令会进入 CLI 交互模式，此时可以与大模型进行对话。

但如果只是需要调用模型 API，执行 `ollama serve` 即可。此时 ollama 会监听本地的 11434 端口，并提供 RESTful API。直接按照[API 文档](https://github.com/ollama/ollama/blob/main/docs/api.md)发起请求，ollama 会加载模型到 GPU 进行计算。

此外，如果一段时间内没有请求，**模型会自动卸载**。

## API

[ollama 局域网访问配置](https://github.com/ollama/ollama/blob/main/docs/faq.md#how-do-i-configure-ollama-server)

In [2]:
import requests
import json

In [17]:
url_generate = "http://172.16.33.242:11434/api/generate"
def get_response(url, data):
    response = requests.post(url, json=data)
    response_dict = json.loads(response.text)
    response_content = response_dict["response"]
    return response_content

data = {
    "model": "llama3.1",
    "prompt": "中国的首都是哪？",
    "stream": False
}


res = get_response(url_generate,data)
print(res)

北京


## LangChain

LangChain 是一个用于开发由大型语言模型(LLMs)支持的应用程序的框架。

In [14]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader

# 指定《奥德赛》在Project Gutenberg上的URL
odyssey_url = "http://whr.085404.xyz/基础指南/Linux docs for the newers.html"

# 实例化WebBaseLoader并加载文档
#loader = WebBaseLoader(odyssey_url)
loader = UnstructuredMarkdownLoader(
    file_path='mk_dataset/RAG/漆黑的魅影攻略.md',
    mode='single'
)
data = loader.load()

In [15]:
print(data)

[Document(metadata={'source': '/home/yzh/code/SELFRec/dataset/RAG/漆黑的魅影攻略.md'}, page_content='title: 漆黑的魅影攻略 date: 2023-08-31 19:16:00 updated: 2023-09-03 10:06:12 categories: - 教程 - 游戏 tags: - game url: /method/pokemon-dark-phantom katex: "false"\n\n神兽\n\n水君\n\n冥想+冲浪+冷冻光线+空气切割\n\n性格：大胆/保守 特性：贮水\n\n努力HP物防，可以考虑分点给特攻。通关比较推荐的类型，冥想三攻，水冰飞的盲点本作就电灯鱼和钢企鹅，打击面很不错，想节约点药钱的可以把一攻换成睡觉。\n\n雷公\n\n紫瑾市精灵中心门口的老头，接任务，拿到地道钥匙 从精灵中心往左下走，到钓鱼老哥那，下水，然后往右到洞口小岛\n\n性格推荐：保守 技能：伏特替换、反射壁、十万伏特和神通力\n\n闪电鸟\n\n地点：卡那兹市上方，流星瀑布最里面\n\n快龙：龙舞+龙之爪/逆鳞+火焰拳+地震， 性格：固执 特性：精神力 努力：物攻速度\n\n技能\n\n学习技能： - 冬雪市，精灵中心往左第二间屋子，技能指导员 - 每次学习需要一个心之鳞片\n\n遗忘技能： - 水静市，超级市场右边的遗忘老人 - 冲浪术必须在背包里另一个精灵也学了的情况下，才能遗忘\n\n金手指\n\ncb： code breaker ，也被称为代码断路器、CB码。格式为：8位十六进制数 4位十六进制数。具体来说，CB码的一行是由单个空格分隔的一个8位十六进制数和一个4位十六进制数，这一行除了两个数字、单个空格和结尾那个看不见的回车符，没有任何其他符号。\n\ngs： gameshark\n\n类型 作用 代码 备注 cb 商店购买物品时，替换为制定物品 83005E12 xxxx xxxx 为物品代码，似乎仅在古辰镇生效 cb 不会遇到野生精灵 820375D4 0000 等价于喷雾 gs 快速升级 020241F0:2710 对战前开启，升到需要等级后即可关闭 gs 开启遇宠（将随机碰到的野生宠物固定为某一个） 0146DCEA 3E32A31D cb 指定遇到

In [34]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 实例化文本分割器，设置每块的字符数
# chunk_overlap参数设置为0意味着文本块之间没有重叠
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10, chunk_overlap=0)

# 分割文档
all_splits = text_splitter.split_documents(data)

In [35]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma

# 实例化嵌入模型
oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")

# 使用文档块创建向量数据库
vectorstore = Chroma.from_documents(documents=all_splits, embedding=oembed)

In [36]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [42]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import  StrOutputParser

template = """Answer the question based only on the following context.
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = Ollama(base_url="http://localhost:11434", model="qwen2")

retriever = vectorstore.as_retriever()
setup_and_retrieval = RunnableParallel(
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
)
chain = setup_and_retrieval | prompt | model | StrOutputParser()

chain.invoke("遗忘老人在哪？")

'根据提供的上下文，“遗忘老人”位于水静市的一个超级市场的右侧。'